In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# Define transforms for data augmentation and normalization
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for batch processing
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:04<00:00, 34199506.38it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torchvision.models as models
import numpy as np
from tqdm import tqdm

# Load pretrained ResNet model and modify it to act as a feature extractor
resnet18 = models.resnet18(pretrained=True)
resnet18 = nn.Sequential(*list(resnet18.children())[:-1])  # Remove the final classification layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

# Function to extract features
def extract_features(dataloader, dataset_name='Dataset'):
    resnet18.eval()  # Set model to evaluation mode
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc=f'Extracting features from {dataset_name}', unit='batch', total=len(dataloader)):
            outputs = resnet18(inputs.to(device)).squeeze()
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.hstack(labels)



# Extract features from train and test set
train_features, train_labels = extract_features(trainloader, 'Train Set')
test_features, test_labels = extract_features(testloader, 'Test Set')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 103MB/s] 
Extracting features from Test Set: 100%|██████████| 10/10 [00:01<00:00,  5.91batch/s]


In [3]:
from sklearn.cluster import KMeans
import numpy as np

# Apply K-means clustering to training set
kmeans = KMeans(n_clusters=10, random_state=42)

train_clusters = kmeans.fit_predict(train_features)

# Assign each test image to its nearest cluster
test_clusters = kmeans.predict(test_features)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_matches(test_feature, cluster_indices, k=50):
    # Get the features of images belonging to the same cluster
    cluster_features = train_features[cluster_indices]
    
    # Compute cosine similarity between the test image and cluster images
    similarities = cosine_similarity(test_feature.reshape(1, -1), cluster_features).flatten()
    
    # Get the top k most similar images
    top_k_indices = np.argsort(similarities)[::-1][:k]
    return cluster_indices[top_k_indices]


# For each test image, find the top 50 matches
top_k_matches = []
for i, test_feature in tqdm(enumerate(test_features), desc='Finding Top-K Matches', unit='image', total=len(test_features)):
    # Find the training images belonging to the same cluster
    cluster_indices = np.where(train_clusters == test_clusters[i])[0]
    
    # Get the top 50 matches based on cosine similarity
    top_k_matches.append(get_top_k_matches(test_feature, cluster_indices))


Finding Top-K Matches: 100%|██████████| 10000/10000 [03:07<00:00, 53.46image/s]


In [5]:
def precision_at_k(true_label, top_k_labels, k):
    top_k = top_k_labels[:k]
    correct = np.sum(top_k == true_label)
    return correct / k

def mean_average_precision(true_label, top_k_labels):
    # Calculate precision at each rank and then compute average precision
    precisions = []
    correct = 0
    for i, label in enumerate(top_k_labels):
        if label == true_label:
            correct += 1
            precisions.append(correct / (i + 1))
    return np.mean(precisions) if precisions else 0

# Evaluate for all test images
precision_10 = []
precision_50 = []
mean_ap = []
for i, matches in tqdm(enumerate(top_k_matches), desc='Evaluating Metrics', unit='image', total=len(top_k_matches)):
    true_label = test_labels[i]
    matched_labels = train_labels[matches]
    
    precision_10.append(precision_at_k(true_label, matched_labels, 10))
    precision_50.append(precision_at_k(true_label, matched_labels, 50))
    mean_ap.append(mean_average_precision(true_label, matched_labels))

# Report final metrics
print(f'Mean Precision@10: {np.mean(precision_10):.4f}')
print(f'Mean Precision@50: {np.mean(precision_50):.4f}')
print(f'Mean Average Precision: {np.mean(mean_ap):.4f}')


Evaluating Metrics: 100%|██████████| 10000/10000 [00:00<00:00, 15351.19image/s]

Mean Precision@10: 0.4224
Mean Precision@50: 0.3724
Mean Average Precision: 0.4547
